In [ ]:
# импортируем необходимые библиотеки
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

In [ ]:
# читаем данные
train = pd.read_csv("titanic_train.csv")
test = pd.read_csv("titanic_test.csv")

In [ ]:
# создаем новые датасеты, в них
# удаляем стобцы, которые физически не могут влиять на выживание
data_train = train.drop(['PassengerId', 'Name', 'Ticket'], axis=1)
data_test = test.drop(['PassengerId', 'Name', 'Ticket'], axis=1)

In [ ]:
# смотрим информацию о датасете для выполнения преобразования данных
# необходимо избавиться от категориальных признаков и пустых значений
data_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB


### Label encoder

In [ ]:
# определяем где в датасете категориальные признаки
list(data_train.select_dtypes(include=['object']).columns)

['Sex', 'Cabin', 'Embarked']

In [ ]:
# смотрим варианты значений категориального признака пол (Sex)
train['Sex'].value_counts()

male      577
female    314
Name: Sex, dtype: int64

In [ ]:
# переводим категориальный признак Sex в дискретный
Sex_embarked_mapping = {'male':0,'female':1}
data_train['Sex'] = data_train['Sex'].map(Sex_embarked_mapping)

In [ ]:
# со столбцом каюта (Cabin) можно выполнить некоторые преобразования
# каждая каюта находится на какой-то палубе и чем ниже палуба, тем меньше шансов выжить
# собственно все было вот так:
from IPython.display import Image
Image(url= "http://data.cyclowiki.org/images/3/38/Titanic_palyb.jpeg")

In [ ]:
# столбец Cabin мало где заполнен, распределим пассажиров

# "отправляем" всех невыживших пассажиров 3-го класса на палубу G
data_train['Cabin'].loc[(data_train['Pclass']==3) & (data_train['Survived']==0)] = 'G'

# "отправляем" всех выживших пассажиров 3-го класса на палубу F
data_train['Cabin'].loc[(data_train['Pclass']==3) & (data_train['Survived']==1)] = 'F'

# "отправляем" всех невыживших пассажиров 2-го класса на палубу E
data_train['Cabin'].loc[(data_train['Pclass']==2) & (data_train['Survived']==0)] = 'E'

# "отправляем" всех выживших пассажиров 2-го класса на палубу D
data_train['Cabin'].loc[(data_train['Pclass']==2) & (data_train['Survived']==1)] = 'D'

# "отправляем" всех невыживших пассажиров 1-го класса на палубу C
data_train['Cabin'].loc[(data_train['Pclass']==1) & (data_train['Survived']==0)] = 'C'

# "отправляем" всех выживших мужчин 1-го класса на палубу B
data_train['Cabin'].loc[(data_train['Pclass']==1) & (data_train['Survived']==1) & (data_train['Sex']==0)] = 'B'
 
# "отправляем" всех выживших женщин 1-го класса на палубу A
data_train['Cabin'].loc[(data_train['Pclass']==1) & (data_train['Survived']==1) & (data_train['Sex']==1)] = 'A'

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [ ]:
# переводим категориальный признак Cabin в дискретный
Cabin_embarked_mapping = {'A':0,'B':1, 'C':2, 'D':3, 'E':4, 'F':5, 'G':6}
data_train['Cabin'] = data_train['Cabin'].map(Cabin_embarked_mapping)

In [ ]:
# заполняем пустые значения столбца Embarked
data_train['Embarked'].loc[(data_train['Embarked'].isna())] ='S'

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [ ]:
# переводим категориальный признак Embarked в дискретный
Embarked_embarked_mapping = {'S':0,'C':1,'Q':2}
data_train['Embarked'] = data_train['Embarked'].map(Embarked_embarked_mapping)

In [ ]:
# заполняем пустые значения столбца Age
data_train['Age'].loc[(data_train['Age'].isna())] = data_train['Age'].mean()

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [ ]:
# проверяем датасет на "избавление" от пустых значений и категориальных признаков
data_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 9 columns):
Survived    891 non-null int64
Pclass      891 non-null int64
Sex         891 non-null int64
Age         891 non-null float64
SibSp       891 non-null int64
Parch       891 non-null int64
Fare        891 non-null float64
Cabin       891 non-null int64
Embarked    891 non-null int64
dtypes: float64(2), int64(7)
memory usage: 62.7 KB


In [ ]:
# отделяем фичи от таргета
y = data_train.Survived
data_train = data_train.drop(['Survived'], axis=1)

In [ ]:
# проводим стандартизацию исходных данных
scaler = preprocessing.StandardScaler()
data_train_ST = scaler.fit_transform(data_train)

In [ ]:
# делим преобразованный pandas датасет на тренировочный и тестовый
X_train, X_test, y_train, y_test = train_test_split(data_train_ST, y, test_size=0.2, random_state=42)

In [ ]:
# создаем параметры KNN для перебора в GridSearchCV
tuned_parameters_KNN = [{'n_neighbors': [1, 2, 5, 10, 50, 100], 'weights': ['distance', 'uniform'], 'metric': ['minkowski', 'euclidean']}]

In [ ]:
# создаем модель на основее KNN и с помощью GridSearchCV
# подбираем оптимальные параметры и обучаем
titanic_model_KNN = GridSearchCV(KNeighborsClassifier(), tuned_parameters_KNN, cv=5)
titanic_model_KNN.fit(X_train,y_train)
titanic_model_KNN.best_params_

{'metric': 'minkowski', 'n_neighbors': 1, 'weights': 'distance'}

In [ ]:
# Считаем accuracy KNN 
np.round(titanic_model_KNN.score(X_test,y_test), 2)

0.91

In [ ]:
# создаем параметры GaussianNB для перебора в GridSearchCV
tuned_parameters_GaussianNB = [{'var_smoothing': [1e-30, 1e-20, 1e-13, 1e-12, 1e-11, 1e-10, 1e-9, 1e-8, 1e-7]}]

In [ ]:
# создаем модель на основее GaussianNB и с помощью GridSearchCV
# подбираем оптимальные параметры и обучаем
titanic_model_GaussianNB = GridSearchCV(GaussianNB(), tuned_parameters_GaussianNB, cv=5)
titanic_model_GaussianNB.fit(X_train,y_train)
titanic_model_GaussianNB.best_params_

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


{'var_smoothing': 1e-30}

In [ ]:
# Считаем accuracy GaussianNB 
np.round(titanic_model_GaussianNB.score(X_test,y_test), 2)

0.8

## One-hot encoding

In [ ]:
# создаем датасеты с ont-hot encoding для столбцов Sex, Cabin, Embarked
OHN_Sex = pd.get_dummies(data_train['Sex'], prefix="Sex", dummy_na=False)
OHN_Cabin = pd.get_dummies(data_train['Cabin'], prefix="Cabin", dummy_na=False)
OHN_Embarked = pd.get_dummies(data_train['Embarked'], prefix="Embarked", dummy_na=False)

In [ ]:
# Собираем закодированные столбцы и исходные данные в один датасет
# удаляем категориальные признаки не в one-hot-encoding
data_train_OHN = pd.concat([data_train, OHN_Sex, OHN_Cabin, OHN_Embarked], axis=1)
data_train_OHN = data_train_OHN.drop(['Sex', 'Cabin', 'Embarked'], axis=1)

In [ ]:
# проводим стандартизацию исходных данных
scaler_OHN = preprocessing.StandardScaler()
data_train_OHN_ST = scaler_OHN.fit_transform(data_train_OHN)

In [ ]:
# делим преобразованный pandas датасет на тренировочный и тестовый
X_train_OHN, X_test_OHN, y_train_OHN, y_test_OHN = train_test_split(data_train_OHN_ST, y, test_size=0.2, random_state=42)

In [ ]:
# создаем параметры KNN для перебора в GridSearchCV
tuned_parameters_KNN_OHN = [{'n_neighbors': [1, 2, 5, 10, 50, 100], 'weights': ['distance', 'uniform'], 'metric': ['minkowski', 'euclidean']}]

In [ ]:
# создаем модель на основее KNN и с помощью GridSearchCV
# подбираем оптимальные параметры и обучаем
titanic_model_KNN_OHN = GridSearchCV(KNeighborsClassifier(), tuned_parameters_KNN_OHN, cv=5)
titanic_model_KNN_OHN.fit(X_train_OHN,y_train_OHN)
titanic_model_KNN_OHN.best_params_

{'metric': 'minkowski', 'n_neighbors': 1, 'weights': 'distance'}

In [ ]:
# Считаем accuracy KNN 
np.round(titanic_model_KNN_OHN.score(X_test_OHN,y_test_OHN), 2)

0.99

In [ ]:
# создаем параметры GaussianNB для перебора в GridSearchCV
tuned_parameters_GaussianNB_OHN = [{'var_smoothing': [1e-30, 1e-20, 1e-13, 1e-12, 1e-11, 1e-10, 1e-9, 1e-8, 1e-7]}]

In [ ]:
# создаем модель на основее GaussianNB и с помощью GridSearchCV
# подбираем оптимальные параметры и обучаем
titanic_model_GaussianNB_OHN = GridSearchCV(GaussianNB(), tuned_parameters_GaussianNB_OHN, cv=5)
titanic_model_GaussianNB_OHN.fit(X_train_OHN,y_train_OHN)
titanic_model_GaussianNB_OHN.best_params_

{'var_smoothing': 1e-30}

In [ ]:
# Считаем accuracy GaussianNB 
np.round(titanic_model_GaussianNB_OHN.score(X_test_OHN,y_test_OHN), 2)

1.0